In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import re
import math
import numpy as np

In [14]:
df = pd.read_csv('tripadvisor_resort_meta.csv',encoding='ISO-8859-1')
df.head()

,resort_meta,url,review_num
0,Ski & Snowboard Areas\rRosa Khutor Ski Resort\...,https://www.tripadvisor.com/Attraction_Review-...,4053
1,Ski & Snowboard Areas\rCatedral Alta Patagonis...,https://www.tripadvisor.com/Attraction_Review-...,4881
2,Ski & Snowboard Areas\rValle Nevado - Ski Reso...,https://www.tripadvisor.com/Attraction_Review-...,3885
3,"Ski & Snowboard Areas\rSnozone\r2,279 reviews\...",https://www.tripadvisor.com/Attraction_Review-...,2279
4,"Ski & Snowboard Areas\rCairnGorm Mountain\r2,2...",https://www.tripadvisor.com/Attraction_Review-...,2267


In [15]:
urls = list(df['url'])

In [16]:
len(urls)

565

In [7]:
def get_url_list(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = requests.get(url, headers=headers)
    search_page = response.text
    soup = BeautifulSoup(search_page,"html5lib")

    # get review number
    eng_revs = soup.find_all("p", {"class": "pagination-details"})
    if eng_revs:
        review_string = eng_revs[0].text
        num_reviews = re.match(r'.+\s(\d+,*\d*)\sreviews',review_string).group(1)
        num_reviews = int(num_reviews.replace(",",""))
        num_pages = math.ceil(num_reviews//10)+1

        # create urls
        review_urls = []

        url_1 = re.match(r'(https://www.tripadvisor.com/Attraction_Review-g\d+-d\d+-Reviews-)(.+html)',url).group(1)
        url_3 = re.match(r'(https://www.tripadvisor.com/Attraction_Review-g\d+-d\d+-Reviews-)(.+html)',url).group(2)
        for i in range(1,num_pages):
            dic = {}    
            url_2 = "or"+str(i*10)+"-"
            dic['url'] = url
            dic['page_url'] = url_1 + url_2 + url_3
            review_urls.append(dic)
        return(review_urls)
    else:
        review_urls = []
        dic = {}
        dic['url'] = url
        dic['page_url'] = url
        review_urls.append(dic)
    return(review_urls)

In [ ]:
# try to grab Eng revss

In [17]:
def eng_rev_num(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = requests.get(url, headers=headers)
    search_page = response.text
    soup = BeautifulSoup(search_page,"html5lib")
    lang_revs = soup.find(class_='ui_column is-3 language').text
    lang_revs = lang_revs.replace('\xa0','')
    lang_revs = lang_revs.replace('('," ")
    lang_revs = lang_revs.replace(')'," ")
    lang_revs = lang_revs.split(" ")
    idx = lang_revs.index('English')+1
    eng_num = lang_revs[idx]
    eng_num = int(eng_num.replace(",",""))
    dic = {}
    dic['url']=url
    dic['eng_rev']=eng_num
    return(dic)

In [18]:
eng_test = []
for i in urls:
    eng_test.append(eng_rev_num(i))

In [20]:
eng_test = pd.DataFrame.from_dict(eng_test)

In [21]:
eng_test.to_csv('eng_revs_num.csv')

In [22]:
eng_test.head()

,eng_rev,url
0,86,https://www.tripadvisor.com/Attraction_Review-...
1,224,https://www.tripadvisor.com/Attraction_Review-...
2,189,https://www.tripadvisor.com/Attraction_Review-...
3,2278,https://www.tripadvisor.com/Attraction_Review-...
4,2175,https://www.tripadvisor.com/Attraction_Review-...


In [23]:
eng_only = eng_test[eng_test.eng_rev>0]

In [ ]:
# create urls for scraping

In [130]:
all_urls = []
for u in urls:
    idx = urls.index(u)
    temp_list = get_url_list(u)
    all_urls.extend(temp_list)     
    temp_df = pd.DataFrame.from_dict(all_urls)
    temp_df.to_csv(f"tripadvisor_reviews/url_batch/url_batch_0.csv")
    print(idx)
    sleep(0.5)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
84
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107


In [132]:
fs = ['all_urls_by_rev_page.csv','tripadvisor_reviews/url_batch/url_batch_0.csv']
for i in fs:
    df = pd.read_csv(i)
    dfs.append(df)

In [133]:
all_urls = pd.concat(dfs)

In [134]:
all_urls.to_csv("final_urls_by_rev_page.csv")